In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

# Load the dataset
df = pd.read_csv('colleges_data.csv')
df.head()

,College Name,Genders Accepted,Total Student Enrollments,Total Faculty,Established Year,University,Courses,Facilities,City,State,Country,College Type,Average Fees,Fee Category,StudentFacultyRatio
0,National Institute of Technology Rourkela,Co-Ed,4811,329,2007,NaN,"B.Tech Mechanical Engineering, B.Tech Computer...","Boys Hostel, Girls Hostel, Gym, Library, Sport...",Rourkela,Odisha,India,Public/Government,350600.0000,High,14.623100
1,Visvesvaraya National Institute of Technology ...,Co-Ed,1071,192,2007,NaN,"B.Tech Computer Science Engineering, B.Tech El...","Boys Hostel, Girls Hostel, Gym, Library, Sport...",Nagpur,Maharashtra,India,Public/Government,273596.6667,High,5.578125
2,Netaji Subhas University of Technology,Co-Ed,3759,101,2018,NaN,"B.Tech Computer Engineering, B.Tech Electronic...","Boys Hostel, Girls Hostel, Gym, Library, Sport...",New Delhi,Delhi,India,Public/Government,352320.0000,High,37.217822
3,Birla Institute of Technology,Co-Ed,4506,284,1955,NaN,"B.Tech Computer Science Engineering, B.Tech El...","Boys Hostel, Girls Hostel, Gym, Library, Sport...",Ranchi,Jharkhand,India,Private,596686.0465,High,15.866197
4,International Institute of Information Technology,Co-Ed,1721,85,2001,NaN,"B.Tech Computer Science and Engineering, M.Tec...","Boys Hostel, Girls Hostel, Gym, Library, Sport...",Hyderabad,Telangana,India,Private,756666.6667,High,20.247059


In [17]:
# Drop the "College Name" column
df = df.drop('College Name', axis=1)

In [27]:
# Separate categorical and numerical columns
categorical_cols = ['Genders Accepted', 'University', 'Courses', 'Facilities', 'City', 'State', 'Country', 'College Type']
numerical_cols = ['Total Student Enrollments', 'Total Faculty', 'Established Year', 'Average Fees', 'StudentFacultyRatio']

# One-hot encode categorical variables
one_hot_encoder = OneHotEncoder()
encoded_categorical_cols = one_hot_encoder.fit_transform(df[categorical_cols])
feature_names = one_hot_encoder.get_feature_names_out(categorical_cols)
encoded_categorical_cols = pd.DataFrame(encoded_categorical_cols.toarray(), columns=feature_names)

In [28]:
# Scale numerical variables
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Concatenate numerical and categorical columns
X = pd.concat([df[numerical_cols], encoded_categorical_cols], axis=1)
y = df['Fee Category']

In [29]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
# Train a random forest classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
# Save the model to a pickle file
with open('admission_model.pkl', 'wb') as f:
    pickle.dump({'model': rfc, 'one_hot_encoder': one_hot_encoder, 'scaler': scaler, 'categorical_cols': categorical_cols, 'numerical_cols': numerical_cols}, f)


In [50]:
# Evaluate the model on the testing set
y_pred = rfc.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9678308823529411
Classification Report:
              precision    recall  f1-score   support

        High       0.99      0.96      0.98       388
         Low       0.97      0.97      0.97       358
      Medium       0.94      0.97      0.96       342

    accuracy                           0.97      1088
   macro avg       0.97      0.97      0.97      1088
weighted avg       0.97      0.97      0.97      1088



In [47]:
# Example to test input
input_data = pd.DataFrame({
    'Genders Accepted': ['Co-Ed'],
    'Total Student Enrollments': [5000],
    'Total Faculty': [200],
    'Established Year': [2010],
    'University': ['NaN'],
    'Courses': ['Course1'],
    'Facilities': ['Facility1'],
    'City': ['New City'],
    'State': ['New State'],
    'Country': ['India'],
    'College Type': ['Private'],
    'Average Fees': [400000],
    'StudentFacultyRatio': [15.0]
})
# Fill NaN values
input_data.fillna({'University': 'Unknown'}, inplace=True)

# One-hot encode categorical variables
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
one_hot_encoder.fit(input_data_categorical_cols)
encoded_input_categorical_cols = one_hot_encoder.transform(input_data_categorical_cols)
feature_names = one_hot_encoder.get_feature_names_out(categorical_cols)
encoded_input_categorical_cols = pd.DataFrame(encoded_input_categorical_cols.toarray(), columns=feature_names)

# Scale numerical variables
input_data_numerical_cols = input_data[numerical_cols]
input_data_numerical_cols = scaler.transform(input_data_numerical_cols)

# Concatenate numerical and categorical columns
input_data = pd.concat([pd.DataFrame(input_data_numerical_cols, columns=numerical_cols), encoded_input_categorical_cols], axis=1)

In [48]:
# Make prediction
def make_predictions(input_data):
    # Encode categorical variables
    input_data_categorical_cols = input_data[categorical_cols]
    encoded_input_categorical_cols = one_hot_encoder.transform(input_data_categorical_cols)
    encoded_input_categorical_cols = pd.DataFrame(encoded_input_categorical_cols.toarray(), columns=feature_names)

    # Scale numerical variables
    input_data_numerical_cols = input_data[numerical_cols]
    scaled_input_numerical_cols = scaler.transform(input_data_numerical_cols)

    # Combine categorical and numerical variables
    input_data_combined = np.concatenate((encoded_input_categorical_cols, scaled_input_numerical_cols), axis=1)

    # Make predictions
    predictions = model.predict(input_data_combined)

    return predictions

In [59]:
# Save the model to a pickle file
with open('admission_model.pkl', 'wb') as f:
    pickle.dump({'model': rfc, 'one_hot_encoder': one_hot_encoder, 'scaler': scaler, 'categorical_cols': categorical_cols, 'numerical_cols': numerical_cols}, f)
